# Revenue Management

## Description

Online revenue management (also known as online stochastic bin packing) considers managing different available resources consumed by different classes of customers in order to maximize revenue.  In this environment, we model multiple types of resources with some initial availability.  At each iteration, the algorithm designer decides in the current time step whether or not to accept customers from a given class.  One customer of a given class comes and arrives to the system, if the agent decides to fulfill their request, they utilize some amount of the different resources and provide an amount of revenue.  At a high level, then, the goal of the agent is to chose which types of customers to accept at every iteration in order to maximize the total revenue.  This requires planning for the scarce resources and ensuring that the agent does not allocate to individuals who will exhaust the remaining resources.

## Model Assumptions
* Customers who are denied are not allowed to purchase resources later even if those resources are available. This did not extend to customer classes, though. A customer may be allowed to purchase resources even if another customer of the same class was denied at an earlier (or later) timestep.

## Environment
### Dynamics

#### State Space
The state space is the set of all possible available resource levels.
$ S = [0, B_1] \times [0, B_2] \times ... \times [0, B_k] $ where $ B_i $ is the maximum availability of resource type $ i $ and $ k $ is the number of resource types.

#### Action Space
The action space is all possible binary vectors of length $ n $ which tells you whether a
customer class is accepted or declined by the company, where n is the number of customer classes.
$ A = {\{0, 1\}}^n $

#### Reward
The one-step reward is the revenue gained from selling resources to the customer class that arrives. If resources are not sold (because the customer is denied or the resources desired are not available), then the reward is zero.

#### Transitions

Given an arrival $ P_t $ of type $ j_t \in [n] $ or $\emptyset$ :
* if $\emptyset$ then $ S_{t+1} = S_t $ with reward $ = 0 $, indicating that no arrivals occurred and so the agent receives no revenue
* if $ j_t $ :
  * if $ a_{j_t} = 0 $ (i.e. algorithm refuses to allocate to that type of customer) then $ S_{t+1} = S_t $ with reward $ = 0 $
  * if $ a_{j_t} = 1 $ and $ S_t - A_{j_t}^T ≥ 0 $ (i.e. budget for resources to satisfy the request) then $ S_{t + 1} = S_t - A_{j_t}^T $ with $ reward = f_{j_t} $


At each time step a customer may or may not arrive. If no customer arrives, then the next state is the same as the current state and the reward is zero. If a customer does arrive they can either be accepted or rejected according to the action taken for the timestep (the action is determined before the arrival of the customer). If the customer is rejected, the next state is the same as the current state and the reward is zero. If the customer is accepted, the resources that they wish to purchase may or may not be available. If they are not available, then the next state is the same as the current state and the reward is zero. If they are available, then the resources purchased are subtracted from the current number available for the next state and the reward is the value determined when initializing the environment for the class of customer that arrived. 

#### Configuration Parameters

* `epLen`: The int number of time steps to run the experiment for.
* `f`: The float array representing the revenue per class.
* `A`: The 2-D float array representing the resource consumption.
* `starting_state`: The float array representing the number of available resources of each type.
* `P`: The float array representing the distribution over arrivals.

### Viewing as an Exo-MDP

We can view this problem set-up as a prime example of an Exo-MDP where the exogenous variable $\xi$ corresponds to the arrival $P_t$ of type $j_t$.  These are independent from the state of the system and algorithm's decisions, and serve as the only unknown in the problem set-up (since we know the true dynamics and reward function of the model as a function of the customer arrival types).

Note here that the exogenous process is assumed to be IID.  As such, we do not need to include it for state information, since subsequent customer arrivals are independent of prior ones.  Hence, the state variable $S_t$ only corresponds to the remaining capacity of each of the resources.

The goal of this code demonstration is to exploit this exogenous problem structure in a variety of ways in algorithm design.

## Heuristic Agents

### Bayes Selector

The Bayes Selector algorithm, at every iteration, solves an optimization problem for the optimal actions based on the current inventory levels and the expected number of future arrival types.  Note that the formulation differs *slightly* from what was discussed in lecture, where the expectation over the exogenous randomness is taken by plugging in the expected number of arrivals into the optimization problem.  However, there is a relationship between these two approaches that we will ignore here for now.


In particular, given the current state $s_t$ denoting the available resource for the $k$ different resource types, we solve the following optimization problem:

$$
    \max \sum_n f_n x_n \\ \text{ s. t. }0 \leq x \leq \mathbb{E}[N_{t}] \\ 0 \leq A x \leq s_t
$$
where $\mathbb{E}[N_{t}]$ is a vector of length $n$ with each element corresponding to the expected number of future arrivals of each type $j$.  The first constraint ensures that the number of allocations to a specific customer type is less than their expected arrival count, and the second is the budgetary constraints.

Letting $x$ be the optimal solution, depending on a rounding flag the action is either taken to be:

-  $a_j = 1$ if $x_j / \mathbb{E}[N_{t}]_j \geq 1/2$, and $0$ otherwise
- $a_j = \text{Bernoulli}(x_j / \mathbb{E}[N_{t}]_i)$

These algorithms arise from recent developments on constant regret algorithms for sequential decision making problems (see [here](https://arxiv.org/abs/1906.06361)).


### Implementing the LP

We will first focus on how to implement this LP in python.  We use the python package [cvxpy](https://www.cvxpy.org/).

In [7]:
import numpy as np

epLen = 4
airline_default_config = {
    'epLen': epLen,
    'f': np.asarray([1., 2.]),
    'A': np.transpose(np.asarray([[2., 3., 2.], [3., 0., 1.]])),
    'starting_state': np.asarray([20/3, 4., 4.]),
    'P': np.asarray([[1/3, 1/3] for _ in range(epLen+1)])
}


In [8]:
import cvxpy as cp
x = cp.Variable(2) # sets up two variables for the two types of customers

In [9]:
objective = cp.Maximize(airline_default_config['f'].T @ x)
# maximize inner product of the allocation and the reward vector

In [10]:
constraints = []

expect_arrivals = np.asarray([5., 6.])
constraints += [0 <= x] # non-negativity constraints
constraints += [x <= expect_arrivals] # cannot allocate more than expected future arrivals, putting in placeholder

constraints += [airline_default_config['A'] @ x <= airline_default_config['starting_state']]
    # budget constraints on the allocations

In [11]:
prob = cp.Problem(objective, constraints)
prob.solve()

4.4444444428596634

This solves the optimization problem.  However, in order to actually get out the solution variables we do:

In [12]:
x.value

array([5.14196485e-09, 2.22222222e+00])

### Hold Up

You should notice that in order to implement this LP we actually need to know the true distribution on the exogenous inputs, which does not exactly mimic the problem set-up which we discussed earlier!  However, what is the most straightforward application of using historical data for this:

- Replace the true expectation on future arrivals with the empirical average from the given exogenous traces

## Goal for Today

In today's code demo we will do two things:

- Implement the Bayes Selector algorithm with *known* exogenous distribution

- Implement the Bayes Selector algorithm with *traces*, where the exogenous distribution is replaced by that with an empirical dataset of samples

The code is contained in the ```or_suite.agents.airline_revenue_management``` folder, with TODO statements highlighted.


After implementing these two algorithms, we will follow up by comparing the performance of three different algorithms:

- Random Algorithm (a base line)

- Bayes Selector (for the true hindsight planning policy $\pi^\dagger$)

- Bayes Selector with exogenous traces (as a simplified example of "Hindsight Learning")

The last option serves as our model of "Hindsight Learning" algorithms.  This is because:

- this problem has a simple enough tabular representation, so there is no need for neural network generalization via the imitation learning reduction
- we can solve for the actual hindsight planning value with traces and use that for the algorithm

The next jupyter notebook will walk through an experiment after implementing the algorithms.  Unfortunately the ```ORSuite``` package is designed for experiments in the ```online``` setting, and not given access to historical traces.  As such, the set-up looks a little strange.